# Hyperparameter optimization

## Importing data and modules

In [2]:
import sys

sys.path.append("../NeuralNetworkClasses")
from extract_from_root import *
from NN_class import *

import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import itertools
import timeit
import uproot

In [6]:
cload = load_tree()
data_path="/lustre/alice/users/csonnab/TPC/NeuralNetworks/TrainingNetworks/LHC18b_FULL_GPU_SUPER_AGG_CUTS_3_smallNet/training_data.root"
data = cload.load(list_ignore=['fRelResoTPC','fNormMultTPC'], path=data_path)

labels = np.array(data[0]).astype('U32')
fit_data = np.array(data[1:]).astype(float)

fit_data = fit_data[fit_data.T[0]!=0]

print(np.shape(fit_data))

(3689856, 10)


In [15]:
np.random.shuffle(fit_data)
X = fit_data[:10000,2:-1]

In [21]:
y = fit_data[:10000,0]*fit_data[:10000,1]

In [22]:
TEST_SIZE = 0.2

#X_train, X_test, y_train, y_test = train_test_split(X_w_ind,y,test_size=TEST_SIZE,shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=TEST_SIZE,shuffle=True)

## Optimizing hyperparameters

In [23]:
import optuna

In [26]:
def objective(trial):
    
    num_layers = trial.suggest_int('num_layers', 0, 30)
    num_neurons_layer = trial.suggest_int('num_neurons_layer', 0, 30)
    
    X_SCALERS = []
    Y_SCALERS = []
    
    H_SIZES = list(itertools.chain(*[[[7,num_neurons_layer]],[[num_neurons_layer,num_neurons_layer]]*num_layers,[[num_neurons_layer,1]]]))
    LAYER_TYPES = list(itertools.chain(*[['fc'],['fc']*(len(H_SIZES)-2), ['fc']]))
    WEIGHT_INIT = nn.init.xavier_normal_
    GAIN = 5/3
    ACTIVATION =list(itertools.chain(*[[nn.ReLU()]*(len(H_SIZES)-1), [nn.Identity()]]))
    
    BATCH_SIZES = [200,50,5]
    N_EPOCHS = 50
    EPOCH_LS = [0,15,40]


    OPTIMIZER = optim.Adam
    WEIGHT_DECAY = 0
    SCHEDULER = optim.lr_scheduler.ReduceLROnPlateau 
    LOSS_FUNCTION = weighted_mse_loss
    LEARNING_RATE = 0.01
    
    NUM_THREADS = 0
    PATIENCE = 5
    FACTOR=0.5
    
    net = General_NN(params = H_SIZES, layer_types = LAYER_TYPES, act_func =ACTIVATION, w_init = WEIGHT_INIT, verbose=True)#, gain=GAIN)
    
    NeuralNet = NN(net)

    data = DataLoading([X_train, y_train], [X_test, y_test], 
                        X_data_scalers=X_SCALERS, y_data_scalers=Y_SCALERS,
                        batch_sizes=BATCH_SIZES, shuffle_every_epoch=True,
                        transformTS=False, transformVS=False)
    
    NeuralNet.training(data, epochs=N_EPOCHS, epochs_ls=EPOCH_LS, weights = False,
                       optimizer=OPTIMIZER,scheduler=SCHEDULER, loss_function=LOSS_FUNCTION, 
                       verbose=True, learning_rate=LEARNING_RATE, patience=PATIENCE, factor=FACTOR)

    return NeuralNet.validation_loss[-1]

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2022-04-27 12:02:54,586] A new study created in memory with name: no-name-bfbfb110-36fc-42e8-86e3-5d76f51ad48b



This is the network structure:

Fully-connected, Input size:  7 , Output size:  22 , activation:  ReLU()
Fully-connected, Input size:  22 , Output size:  1 , activation:  Identity()

Preparing training data...
No transformation is performed on training data!
Training data transformed.
Continuing with validation data...

No transformation is performed on validation data!
Validation data transformed.
Duration: 0.021 s

Memory allocated:  Training data: 0.256 MiB, Validation data: 0.064 MiB

Data is loaded, Training can begin!


Running on CPU
64 CPU threads

Epoch  1 / 50 | Batch-size:  200 , Validation loss:  0.055472 , Training loss:  0.598322 , Execution time:  0.154 s
Epoch  2 / 50 | Batch-size:  200 , Validation loss:  0.020025 , Training loss:  0.040414 , Execution time:  0.101 s
Epoch  3 / 50 | Batch-size:  200 , Validation loss:  0.012065 , Training loss:  0.015746 , Execution time:  0.099 s
Epoch  4 / 50 | Batch-size:  200 , Validation loss:  0.009921 , Training loss:  0.010678

[I 2022-04-27 12:03:21,061] Trial 0 finished with value: 0.005748252384364605 and parameters: {'num_layers': 0, 'num_neurons_layer': 22}. Best is trial 0 with value: 0.005748252384364605.


Epoch  50 / 50 | Batch-size:  5 , Validation loss:  0.005748 , Training loss:  0.005591 , Execution time:  1.975 s

Training finished!


This is the network structure:

Fully-connected, Input size:  7 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, Input size:  19 , Output size:  19 , activation:  ReLU()
Fully-connected, I

KeyboardInterrupt: 